![imagenes](logo.png)

In [ ]:
import pandas as pd
import numpy as np
from siuba import *
from siuba.dply.vector import *
import os

In [ ]:
## Nos dirigimos a nuestra carpeta de trabajo
os.chdir("C:\\Users\\hp master\\OneDrive\\Escritorio\\delitos")

In [ ]:
## Observamos qué tenemos en esa carpeta
os.listdir()

In [ ]:
## Leemos las tablas de delitos
delitos = [pd.read_csv(x) for x in os.listdir() if x.startswith("delitos")]

In [ ]:
## Mostramos la primer tabla
delitos[0]

In [ ]:
## Leemos la tabla descriptor_delitos y la mostramos
descriptor_delitos = pd.read_csv("descriptor_delitos.csv",encoding="latin-1")
descriptor_delitos
descriptor_delitos.columns

In [ ]:
## Leemos la tabla descriptor_entidades y la mostramos
descriptor_entidades = pd.read_csv("descriptor_entidades.csv",encoding="latin-1")
descriptor_entidades
descriptor_entidades.columns

In [ ]:
(delitos[0] >>
    mutate(Año = 0 + 2012) >>
    gather("cod_del",
           "total",
           _[1:16]) >>
    left_join(_,descriptor_entidades, on = "entidad") >>
    mutate(cod_del = _.cod_del.astype("int"), total = _.total.astype("Int64")) >>
    left_join(_,descriptor_delitos, on = "cod_del") >>
    select(_.Año, _.Ent_cod == _.entidad,_.Entidad,_.Delito, _.Total == _.total)
    )

In [ ]:
def ordenado(x):
    auxiliar = (delitos[x] >>
                    mutate(Año = x + 2012) >>
                    gather("cod_del",
                           "total",
                           _[1:16]) >>
                    left_join(_,descriptor_entidades, on = "entidad") >>
                    mutate(cod_del = _.cod_del.astype("int"), total = _.total.astype("Int64")) >>
                    left_join(_,descriptor_delitos, on = "cod_del") >>
                    select(_.Año, _.Ent_cod == _.entidad,_.Entidad,_.Delito, _.Total == _.total)
    )
    return auxiliar
    

In [ ]:
delitos_tidy = pd.concat([ordenado(x) for x in range(0,10)])

In [ ]:
delitos_tidy

In [ ]:
(delitos_tidy >> 
    group_by(_.Año,_.Entidad) >> 
    summarize(Total_delitos_cometidos = _.Total.sum()) >>
    arrange(-_.Total_delitos_cometidos) >>
    head(10)
)